<h1><span style="color:red">Simple Variable Calculations</span></h1>

### This sample notebook reads numeric variables from a survey dataset and lets users compute a new numeric variable, add it to a new survey version, and publish the survey to the user's surveys gallery

## 1. Retrieve survey parameters from the URL

In [167]:
%%javascript
function getQueryStringValue (key)
{  
    return unescape(window.location.search.replace(new RegExp("^(?:.*[&\\?]" + escape(key).replace(/[\.\+\*]/g, "\\$&") + "(?:\\=([^&]*))?)?.*$", "i"), "$1"));
}
IPython.notebook.kernel.execute("survey_url='".concat(getQueryStringValue("surveyurl")).concat("'"));
IPython.notebook.kernel.execute("views='".concat(getQueryStringValue("views")).concat("'"));
IPython.notebook.kernel.execute("view='".concat(getQueryStringValue("view")).concat("'"));
IPython.notebook.kernel.execute("user='".concat(getQueryStringValue("user")).concat("'"));
IPython.notebook.kernel.execute("csv_file='".concat(getQueryStringValue("csv")).concat("'")); 
IPython.notebook.kernel.execute("dzc_file='".concat(getQueryStringValue("dzc")).concat("'")); 
IPython.notebook.kernel.execute("params='".concat(getQueryStringValue("params")).concat("'")); 
IPython.notebook.kernel.execute("active_object='".concat(getQueryStringValue("activeobject")).concat("'")); 
IPython.notebook.kernel.execute("full_notebook_url='" + window.location + "'"); 

<IPython.core.display.Javascript object>

## 2. Read the survey file and extract numeric variables

In [168]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import pandas as pd
from IPython.display import Markdown, display
def printmd(string):
    display(Markdown(string))

absolutePath = "../../temp_csvs/"

# read the csv file
file = open(absolutePath + csv_file, encoding="latin-1")
# print(absolutePath + csv_file)
df = pd.read_csv(file)

# create a list of variable names
variables_df = pd.DataFrame({'varname':df.columns})
printmd("<b><span style='color:red'>All variables in the survey file:</span></b>")
print(variables_df.varname.values)

# create a dictionary of #number variables with abbreviated and full variable names 
var_list = {n[:n.index('#')]:n for n in variables_df.varname.values if '#number' in n}
printmd("<b><span style='color:red'>Numeric variables:</span></b>")
print(var_list)


<b><span style='color:red'>All variables in the survey file:</span></b>

['Index#number #hidden' 'ZipCode' 'BreedName' 'weight#number' '#img'
 'median_income#number' 'FELONY#number' 'MISDEMEANOR#number'
 'num_crimes#number' 'price#number' '#name' 'geometry#hiddenmore']


<b><span style='color:red'>Numeric variables:</span></b>

{'median_income': 'median_income#number', 'Index': 'Index#number #hidden', 'weight': 'weight#number', 'FELONY': 'FELONY#number', 'MISDEMEANOR': 'MISDEMEANOR#number', 'price': 'price#number', 'num_crimes': 'num_crimes#number'}


## 3. Define a new variable using the form ##

In [169]:
# Define a new Variable
a = widgets.Dropdown(options=var_list.keys())
b = widgets.Dropdown(options=['/','*','+','-'])
c = widgets.Dropdown(options=var_list.keys())
ui = widgets.VBox([a, b, c])
def f(a, b, c):
    return ((a, b, c))

formula = widgets.interactive_output(f, {'a': a, 'b': b, 'c': c})

display(ui, formula)

printmd("<b><span style='color:red'>Select variables and the operation, then run the next cell</span></b>")


Output()

<b><span style='color:red'>Select variables and the operation, then run the next cell</span></b>

## 4. Edit variable name as needed

In [170]:
# Give a Name to the New Variable
def f(Var_Name):
    return Var_Name
newvar = interact(f, Var_Name=a.value + b.value + c.value +'#number');

printmd("<b><span style='color:red'>After defining variable name hit Enter, then run the next cell</span></b>")



interactive(children=(Text(value='weight/MISDEMEANOR#number', description='Var_Name'), Output()), _dom_classes…

<b><span style='color:red'>After defining variable name hit Enter, then run the next cell</span></b>

## 5. Compute the new variable and format it for SuAVE

In [171]:
try:
    if b.value == '/':
        df[newvar.widget.result] = df[var_list[a.value]] / df[var_list[c.value]]
    elif b.value == '*':
        df[newvar.widget.result] = df[var_list[a.value]] * df[var_list[c.value]]
    elif b.value == '+':
        df[newvar.widget.result] = df[var_list[a.value]] + df[var_list[c.value]]
    elif b.value == '-':
        df[newvar.widget.result] = df[var_list[a.value]] - df[var_list[c.value]]
        
# make sure there are no illegal NaN type values in this #number variable
    df[newvar.widget.result].fillna('',inplace=True)
    df[newvar.widget.result] = pd.to_numeric(df[newvar.widget.result], errors='coerce', downcast='float')
    df[newvar.widget.result] = df[newvar.widget.result].apply(lambda x: '{:.6f}'.format(x))
    df.replace(['None', 'nan'], pd.np.nan, inplace=True)
    printmd("<b><span style='color:red'>New variable computed</span></b>")

except:
    printmd("<b><span style='color:red'>!! Cannot compute. One or both variables contain non-numeric values!!</span></b>")



<b><span style='color:red'>New variable computed</span></b>

## 6. Save the new version of CSV file, and give a name to new survey

In [172]:
# new filename
new_file = absolutePath + csv_file[:-4]+'_v1.csv'
printmd("<b><span style='color:red'>A new temporary file will be created at: </span></b>")
print(new_file)
df.to_csv(new_file, index=None)

<b><span style='color:red'>A new temporary file will be created at: </span></b>

../../temp_csvs/zaslavsk_hetsvi2_v1.csv


In [173]:
#Input survey name

from IPython.display import display
input_text = widgets.Text()
output_text = widgets.Text()

def bind_input_to_output(sender):
    output_text.value = input_text.value

# Tell the text input widget to call bind_input_to_output() on submit
input_text.on_submit(bind_input_to_output)

printmd("<b><span style='color:red'>Input survey name here, press Enter, and then run the next cell:</span></b>")
# Display input text box widget for input
display(input_text)

display(output_text)


<b><span style='color:red'>Input survey name here, press Enter, and then run the next cell:</span></b>

Text(value='')

Text(value='')

In [174]:
#Print survey name
survey_name = output_text.value
printmd("<b><span style='color:red'>Survey Name is: </span></b>" + survey_name)



<b><span style='color:red'>Survey Name is: </span></b>dod1

### 7. Generate the survey and create survey URL

In [175]:
referer = survey_url.split("/main")[0] +"/"
upload_url = referer + "uploadCSV"
new_survey_url_base = survey_url.split(user)[0]

import requests
import re
csv = {"file": open(new_file, "rb")}
upload_data = {
    'name': input_text.value,
    'dzc': dzc_file,
    'user':user
}
headers = {
    'User-Agent': 'suave user agent',
    'referer': referer
}

r = requests.post(upload_url, files=csv, data=upload_data, headers=headers)

if r.status_code == 200:
    printmd("<b><span style='color:red'>New survey created successfully</span></b>")
    regex = re.compile('[^0-9a-zA-Z_]')
    s_url = survey_name
    s_url =  regex.sub('_', s_url)

    url = new_survey_url_base + user + "_" + input_text.value + ".csv" + "&views=" + views + "&view=" + view
    print(url)
    printmd("<b><span style='color:red'>Click the URL to open the new survey</span></b>")
else:
    printmd("<b><span style='color:red'>Error creating new survey. Check if a survey with this name already exists.</span></b>")
    printmd("<b><span style='color:red'>Reason: </span></b>"+ str(r.status_code) + " " + r.reason)



<b><span style='color:red'>New survey created successfully</span></b>

https://suave2.tirebiter.org/main/file=zaslavsk_dod1.csv&views=1110101&view=grid


<b><span style='color:red'>Click the URL to open the new survey</span></b>